In [ ]:
import sqlparse
from difflib import SequenceMatcher
import glob
import pandas as pd
pd.set_option('max_colwidth', 400)
import re
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
def calculate_query_similarity_with_sequence_matcher():

    migrated_queries = read_in_excel_files("../Query Processing/1_migrated_excel_queries/*.xlsx")
    migrated_queries_preprocessed = preprocess_query_df(migrated_queries, "migrated")

    new_queries = read_in_excel_files("../Query Processing/2_new_excel_queries/*.xlsx")
    new_queries_preprocessed = preprocess_query_df(new_queries, "new")

    return calculate_query_similarity(migrated_queries_preprocessed, new_queries_preprocessed)

In [16]:
res = calculate_query_similarity_with_sequence_matcher()

2025-03-31 11:01:26,836 - INFO - Successfully read 3 excel file(s) from ../Query Processing/1_migrated_excel_queries/*.xlsx.
2025-03-31 11:01:26,838 - INFO - Started preprocess_query_df for migrated queries.
2025-03-31 11:01:47,340 - INFO - Preprocessing of migrated queries successful.
2025-03-31 11:01:47,416 - INFO - Successfully read 1 excel file(s) from ../Query Processing/2_new_excel_queries/*.xlsx.
2025-03-31 11:01:47,417 - INFO - Started preprocess_query_df for new queries.
2025-03-31 11:03:00,306 - INFO - Preprocessing of new queries successful.
2025-03-31 11:03:00,308 - INFO - Started calculate_query_similarity.
2025-03-31 11:13:17,213 - INFO - Query similarity calculation completed.


In [20]:
res.to_excel("./Results/similarity_report_sequence_matcher.xlsx", index=False)

In [11]:
res

,Report Name,migrated_query,Report Path,Folder Path,Product Name,Datasource,new_query,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,similarity,row_number
97090,CDC TRANSTYPE [Handling and Storage.Op.0551],"SELECT ""SUVOL"", TRIM(BOTH FROM ""PARTREV""), SUBSTR(""PARTNO"", 1, 8), ""DATREG"", TRIM(BOTH FROM ""LOGGUSER""), ""X08RCODE"", ""X08QUANT"", TRIM(BOTH FROM ""PARTNO"") FROM (SELECT ""DATREG"", ""LOGGUSER"", ""PARTNO"", ""PARTREV"", ""X08QUANT"", ""X08RCODE"" FROM ""X08T3"" WHERE TRIM(BOTH FROM ""DIVCODE"") NOT IN (""055"")) ""X08T37"" INNER JOIN (SELECT ""PARTNO"", ""PARTREV"", ""SUVOL"" FROM ""L62T1"" WHERE TRIM(BOTH FROM ""DIVCODE"") ...",,,CDC Current Week Adjustments,,"SELECT ""DATE1"", ""TRANSACTION_GROUP"", ""TRANS_TYPE"", ""DIVISION"", ""ARTICLE"", ""SUPPLIER"", ""ADJ_QTY"", ""LOG_ID"", ""BUFFER_MHA"", ""PICK_MHA"", ""PALLET_CLASS"", ""CARRNO"", ""PALLET_ID"", ""PREVIOUS_TOUCH"", ""LAST_USER"", TO_NUMBER(""ADJ_QTY"") ""C15"" FROM (SELECT DISTINCT ""DATE"", ""TRANSACTION GROUP"" ""TRANSACTION_GROUP"", ""TRANS TYPE"" ""TRANS_TYPE"", ""DIVISION"", ""ARTICLE"", ""SUPPLIER"", ""ADJ QTY"" ""ADJ_QTY"", ""LOG ID"" ""LO...",,,,,,,,,0.075589,1
97685,CDC TRANSTYPE [Handling and Storage.Op.0551],"SELECT ""SUVOL"", TRIM(BOTH FROM ""PARTREV""), SUBSTR(""PARTNO"", 1, 8), ""DATREG"", TRIM(BOTH FROM ""LOGGUSER""), ""X08RCODE"", ""X08QUANT"", TRIM(BOTH FROM ""PARTNO"") FROM (SELECT ""DATREG"", ""LOGGUSER"", ""PARTNO"", ""PARTREV"", ""X08QUANT"", ""X08RCODE"" FROM ""X08T3"" WHERE TRIM(BOTH FROM ""DIVCODE"") NOT IN (""055"")) ""X08T37"" INNER JOIN (SELECT ""PARTNO"", ""PARTREV"", ""SUVOL"" FROM ""L62T1"" WHERE TRIM(BOTH FROM ""DIVCODE"") ...",,,CDC Current Week Adjustments,,"SELECT ""DATE1"", ""TRANSACTION_GROUP"", ""TRANS_TYPE"", ""DIVISION"", ""ARTICLE"", ""SUPPLIER"", ""ADJ_QTY"", ""LOG_ID"", ""BUFFER_MHA"", ""PICK_MHA"", ""PALLET_CLASS"", ""CARRNO"", ""PALLET_ID"", ""PREVIOUS_TOUCH"", ""LAST_USER"", TO_NUMBER(""ADJ_QTY"") ""C15"" FROM (SELECT DISTINCT ""DATE"", ""TRANSACTION GROUP"" ""TRANSACTION_GROUP"", ""TRANS TYPE"" ""TRANS_TYPE"", ""DIVISION"", ""ARTICLE"", ""SUPPLIER"", ""ADJ QTY"" ""ADJ_QTY"", ""LOG ID"" ""LO...",,,,,,,,,0.075388,2
98280,CDC TRANSTYPE [Handling and Storage.Op.0551],"SELECT ""SUVOL"", TRIM(BOTH FROM ""PARTREV""), SUBSTR(""PARTNO"", 1, 8), ""DATREG"", TRIM(BOTH FROM ""LOGGUSER""), ""X08RCODE"", ""X08QUANT"", TRIM(BOTH FROM ""PARTNO"") FROM (SELECT ""DATREG"", ""LOGGUSER"", ""PARTNO"", ""PARTREV"", ""X08QUANT"", ""X08RCODE"" FROM ""X08T3"" WHERE TRIM(BOTH FROM ""DIVCODE"") NOT IN (""055"")) ""X08T37"" INNER JOIN (SELECT ""PARTNO"", ""PARTREV"", ""SUVOL"" FROM ""L62T1"" WHERE TRIM(BOTH FROM ""DIVCODE"") ...",,,CDC Current Week Adjustments,,"SELECT ""DATE1"", ""TRANSACTION_GROUP"", ""TRANS_TYPE"", ""DIVISION"", ""ARTICLE"", ""SUPPLIER"", ""ADJ_QTY"", ""LOG_ID"", ""BUFFER_MHA"", ""PICK_MHA"", ""PALLET_CLASS"", ""CARRNO"", ""PALLET_ID"", ""PREVIOUS_TOUCH"", ""LAST_USER"", TO_NUMBER(""ADJ_QTY"") ""C15"" FROM (SELECT DISTINCT ""DATE"", ""TRANSACTION GROUP"" ""TRANSACTION_GROUP"", ""TRANS TYPE"" ""TRANS_TYPE"", ""DIVISION"", ""ARTICLE"", ""SUPPLIER"", ""ADJ QTY"" ""ADJ_QTY"", ""LOG ID"" ""LO...",,,,,,,,,0.075388,3
98875,CDC TRANSTYPE [Handling and Storage.Op.0551],"SELECT ""SUVOL"", TRIM(BOTH FROM ""PARTREV""), SUBSTR(""PARTNO"", 1, 8), ""DATREG"", TRIM(BOTH FROM ""LOGGUSER""), ""X08RCODE"", ""X08QUANT"", TRIM(BOTH FROM ""PARTNO"") FROM (SELECT ""DATREG"", ""LOGGUSER"", ""PARTNO"", ""PARTREV"", ""X08QUANT"", ""X08RCODE"" FROM ""X08T3"" WHERE TRIM(BOTH FROM ""DIVCODE"") NOT IN (""055"")) ""X08T37"" INNER JOIN (SELECT ""PARTNO"", ""PARTREV"", ""SUVOL"" FROM ""L62T1"" WHERE TRIM(BOTH FROM ""DIVCODE"") ...",,,CDC Current Week Adjustments,,"SELECT ""DATE1"", ""TRANSACTION_GROUP"", ""TRANS_TYPE"", ""DIVISION"", ""ARTICLE"", ""SUPPLIER"", ""ADJ_QTY"", ""LOG_ID"", ""BUFFER_MHA"", ""PICK_MHA"", ""PALLET_CLASS"", ""CARRNO"", ""PALLET_ID"", ""PREVIOUS_TOUCH"", ""LAST_USER"", TO_NUMBER("

##### `read_in_excel_files`

In [4]:
def read_in_excel_files(PATH):
    try:
        excel_data = glob.glob(PATH)

        dataframes = [pd.read_excel(data, engine="openpyxl") for data in excel_data]
        query_df = pd.concat(dataframes, ignore_index=True)

        logging.info(f"Successfully read {len(excel_data)} excel file(s) from {PATH}.")

    except Exception as e:
        logging.error(f"Error in reading excel files: {e}")

    return query_df

##### `preprocess_query_df`

In [ ]:
def preprocess_query_df(query_df, QUERY_IDENTIFIER):
    logging.info(f"Started {preprocess_query_df.__name__} for {QUERY_IDENTIFIER} queries.")

    query_df = query_df.dropna(subset=["SQL"]).fillna("")

    query_df["SQL"] = (query_df["SQL"]
                       .str.replace('ê', 'e').str.replace('é', 'e').str.replace('è', 'e').str.replace('à', 'a').str.replace('ç', 'c')
                       .str.replace('ô', 'o').str.replace('û', 'u').str.replace('ù', 'u').str.replace('î', 'i').str.replace('ï', 'i')
                       .str.replace('â', 'a').str.replace('ä', 'a').str.replace('ö', 'o').str.replace('ü', 'u').str.replace('ÿ', 'y')
                       .str.replace('ñ', 'n').str.replace('É', 'E').str.replace('È', 'E').str.replace('À', 'A').str.replace('Ç', 'C')
                       .str.replace('Ô', 'O').str.replace('Û', 'U').str.replace('Ù', 'U').str.replace('Î', 'I').str.replace('Ï', 'I')
                       .str.replace('Â', 'A').str.replace('Ä', 'A').str.replace('Ö', 'O').str.replace('Ü', 'U').str.replace('Ÿ', 'Y')
                       .str.replace('Ñ', 'N')
    )

    with_pattern = r'WITH\s+"\w+"\s+AS\s*\(.*?\)\s*SELECT'
    pattern_2 = r'"[^"]*"\.'
    pattern_3 = r'\w+\.'

    for index, query in enumerate(query_df["SQL"]):
        formatted_query = query.upper()
        formatted_query = sqlparse.format(formatted_query, reindent=True, keyword_case='upper', strip_comments=True).strip()
        formatted_query = formatted_query.replace("'", '"').replace("\n", " ")
        formatted_query = re.sub(with_pattern, 'SELECT', formatted_query, flags=re.DOTALL)
        formatted_query = re.sub(pattern_2, '', formatted_query)
        formatted_query = re.sub(pattern_3, '', formatted_query)
        formatted_query = " ".join(formatted_query.split())
        formatted_query = re.sub(r'"(\w+)" AS "\1"', r'"\1"', formatted_query)
        formatted_query = re.sub(r'("\w+")\s+"\w+"', r'\1', formatted_query)

        query_df.at[index, 'SQL'] = formatted_query

    logging.info(f"Preprocessing of {QUERY_IDENTIFIER} queries successful.")

    return query_df

##### `calculate_query_similarity`

In [15]:
def calculate_query_similarity(migrated_queries_preprocessed, new_queries_preprocessed):
    logging.info(f"Started {calculate_query_similarity.__name__}.")

    cross_join_df = migrated_queries_preprocessed.assign(key=1).merge(new_queries_preprocessed.assign(key=1), on='key').drop('key', axis=1)
    cross_join_df["similarity"] = cross_join_df.apply(lambda row: SequenceMatcher(None, row["SQL_x"], row["SQL_y"]).ratio(), axis=1)
    cross_join_df.rename(columns={"SQL_x": "migrated_report_sql", "SQL_y": "new_report_sql"}, inplace=True)

    cross_join_df = cross_join_df.sort_values(by=['Product Name', 'similarity'], ascending=[True, False])
    cross_join_df["row_number"] = cross_join_df.groupby("Product Name").cumcount() + 1
    cross_join_df = cross_join_df[cross_join_df["row_number"] < 4]

    logging.info(f"Query similarity calculation completed.")

    return cross_join_df[["Product Name", "Report Name", "similarity", "new_report_sql", "migrated_report_sql", "row_number"]].rename(columns={"Report Name": "migrated_report_name", "Product Name": "new_report_name"})

In [ ]:
def calculate_query_similarity(migrated_queries_preprocessed, new_queries_preprocessed):
    logging.info(f"Started {calculate_query_similarity.__name__}.")

    cross_join_df = migrated_queries_preprocessed.assign(key=1).merge(new_queries_preprocessed.assign(key=1), on='key').drop('key', axis=1)
    cross_join_df["similarity"] = cross_join_df.apply(lambda row: SequenceMatcher(None, row["SQL_x"], row["SQL_y"]).ratio(), axis=1)
    cross_join_df.rename(columns={"SQL_x": "migrated_report_sql", "SQL_y": "new_report_sql"}, inplace=True)

    cross_join_df = cross_join_df.sort_values(by=['Product Name', 'similarity'], ascending=[True, False])
    cross_join_df["row_number"] = cross_join_df.groupby("Product Name").cumcount() + 1

    cross_join_df = cross_join_df[cross_join_df["row_number"] < 4]

    logging.info(f"Query similarity calculation completed.")

    result_df = cross_join_df[["Product Name", "Report Name", "similarity", "new_report_sql", "migrated_report_sql", "row_number"]]
    result_df = result_df.rename(columns={"Report Name": "migrated_report_name", "Product Name": "new_report_name"})

    return result_df